# Plot paired and unpaired synapses

In [1]:
# Load up all of the libraries we need

import copy

from deriva.core import HatracStore, ErmrestCatalog, get_credential
from synspy.analyze.pair import SynapticPairStudy

import synapse_utils

import pandas as pd
import numpy as np

import plotly as py
import plotly.graph_objs as go


# configure plot.ly to run in a notebook.
py.offline.init_notebook_mode(connected=True)

We are going to pull the synapse data down directly from the syapse server.  In order to do this we have to make a connection to the server at synapse.isrd.isi.edu You need to make sure you have logged in using the DERIVA login agent first

In [2]:
credential = get_credential("synapse.isrd.isi.edu")
ermrest_catalog = ErmrestCatalog('https', 'synapse.isrd.isi.edu', 1, credential)
hatrac_store = HatracStore('https', 'synapse.isrd.isi.edu', credentials=credential)

Lets start by getting a list of all of the studies currently in the server. We will end up with a list of studies, where each study has been classified by the study type based on the protocol that was followed. 

In [3]:
# Get the current list of studies from the server.
study_entities = synapse_utils.get_synapse_studies()

print('Identified %d studies' % len(study_entities))

protocol_types = {
    'PrcDsy20160101A': 'aversion',
    'PrcDsy20170613A': 'conditioned',
    'PrcDsy20170615A': 'unconditioned',
    'PrcDsy20170613B': 'fullcycle-control',
    'PrcDsy20171030A': 'groundtruth-control',
    'PrcDsy20171030B': 'interval-groundtruth-control'
}

studies={}
for i in study_entities:
    studies[i['Study']] = {}
    studies[i['Study']]['study'] = i['Study']
    if protocol_types[i['Protocol']] == 'aversion':
        studies[i['Study']]['type'] = 'learner' if i['Learner'] == True else 'nonlearner'
    else:
        studies[i['Study']]['type'] = protocol_types[i['Protocol']]
        

HTTPError: 401 Client Error: Unauthorized for url: https://synapse.isrd.isi.edu/ermrest/catalog/1/schema

Lets make some convienent subsets of studies. These are just lists of study names.

In [ ]:
learners= [x for x,y in studies.items() if y['type'] == 'learner' ]
nonlearners = [ x for x,y in studies.items() if y['type'] == 'nonlearner' ]
all = [x for x in studies]

print('{0} learners'.format(len(learners)))
print('{0} non-learners'.format(len(nonlearners)))

Pick a study that we want to look at and set the parameters for the sweep.

In [ ]:
syn_pair_radii = (2.0, 3.0, 4.0) # maximum search radii
syn_dx_core_ratio = None  # turn off 4D nearest-neighbor
syn_core_max_ratio = None # turn off intensity ratio threshold

Now calculate the distances and save the results into dataframes. We are going to orignize things so that we have a copy of studies for each radii that we have, and put them in a top level dictionary.

In [ ]:
# We will have one dictonary of studies per radius
syn_pairs = {r: {} for r in syn_pair_radii}

# Now go through the studies and get the ones you want
study_list = learners + nonlearners

for syn_study_id in study_list:
    print('Processing study {0}'.format(syn_study_id))
    study = SynapticPairStudy.from_study_id(ermrest_catalog, syn_study_id)
    study.retrieve_data(hatrac_store)
        
    s1_to_s2, s2_to_s1 = study.syn_pairing_maps(syn_pair_radii, syn_dx_core_ratio, syn_core_max_ratio)

    # get results for different radii, store them in a dictonary of pandas
    for i,r in enumerate(syn_pair_radii):
        syn_pairs[r][syn_study_id] = copy.deepcopy(studies[syn_study_id])
        r_study = syn_pairs[r][syn_study_id]
        
        unpaired1 = study.get_unpaired(s1_to_s2[i,:], study.s1)
        unpaired2 = study.get_unpaired(s2_to_s1[i,:], study.s2)
        paired1, paired2 = study.get_pairs(s1_to_s2[i,:], study.s1, study.s2)
        
        r_study['radius'] = r
        r_study['study'] = syn_study_id
    
        r_study['unpaired_before'] =  \
            pd.DataFrame(unpaired1[:,0:5], columns=['x','y','z','core','hollow'])
        r_study['unpaired_after'] = \
            pd.DataFrame(unpaired2[:,0:5], columns=['x','y','z','core','hollow'])
        r_study['paired_before'] = \
            pd.DataFrame(paired1[:,0:5], columns=['x','y','z','core','hollow'])
        r_study['paired_after'] = \
            pd.DataFrame(paired2[:,0:5], columns=['x','y','z','core','hollow'])

<H1> Compute some basic Statistics

In [ ]:
# Set what radius/studies you want to have....
study_list = syn_pairs[3.0]

cols = ['ID', 'Type', 'Radius', 'Unpaired Before', 'Unpaired After','Paired Before','Paired After']
c1, c2, c3, c4, c5, c6, c7 = [],[],[],[],[],[],[]

for id,d in study_list.items():
    c1.append(id)
    c2.append(d['type'])
    c3.append(d['radius'])
    c4.append(len(d['unpaired_before'].index))
    c5.append(len(d['unpaired_after'].index))
    c6.append(len(d['paired_before'].index))
    c7.append(len(d['paired_before'].index))
    
trace = go.Table(
    header=dict(values=cols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[c1, c2, c3, c4, c5, c6, c7],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

data = [trace] 
py.offline.iplot(data, filename = 'statistics_table')

<H1> Plot histograms of pair distance for all of the studies

In [ ]:
sl = pairs[max(pairs)]

for k, v in pairs[max(pairs)].items():
    before = v['paired_before']
    after = v['paired_after']
    pair_distance = np.sqrt((before['x'] - after['x'])**2 +
                        (before['y'] - after['y'])**2 +
                        (before['z'] - after['z'])**2)
                
hist_data = go.Histogram(x=pair_distance, name='Radias {0}'.format(r))

hist_layout = go.Layout(
    title='Pair Histogram Radius {0}'.format(r),
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data= [hist_data], layout=hist_layout)

py.offline.iplot(hist_fig, filename='pair-distance')

<H1> Unpaired Centroids

Compute the centroids of the unpaired syapses before/after.  Also find the vector between the before and after centroids.  

In [ ]:
# Select the radius that we want to calculate for
sl = syn_pairs[3.0]

# calculate centroids
lbx = [v['unpaired_before']['x'].mean() for k,v in sl.items() if v['type'] == 'learner']
lby = [v['unpaired_before']['y'].mean() for k,v in sl.items() if v['type'] == 'learner']
lbz = [v['unpaired_before']['z'].mean() for k,v in sl.items() if v['type'] == 'learner']

lax = [v['unpaired_after']['x'].mean() for k,v in sl.items() if v['type'] == 'learner']
lay = [v['unpaired_after']['y'].mean() for k,v in sl.items() if v['type'] == 'learner']
laz = [v['unpaired_after']['z'].mean() for k,v in sl.items() if v['type'] == 'learner']

nbx = [v['unpaired_before']['x'].mean() for k,v in sl.items() if v['type'] == 'nonlearner']
nby = [v['unpaired_before']['y'].mean() for k,v in sl.items() if v['type'] == 'nonlearner']
nbz = [v['unpaired_before']['z'].mean() for k,v in sl.items() if v['type'] == 'nonlearner']

nax = [v['unpaired_after']['x'].mean() for k,v in sl.items() if v['type'] == 'nonlearner']
nay = [v['unpaired_after']['y'].mean() for k,v in sl.items() if v['type'] == 'nonlearner']
naz = [v['unpaired_after']['z'].mean() for k,v in sl.items() if v['type'] == 'nonlearner']

# calculate the vector of the centroid differences.
lcx = [ v[0] - v[1] for v in zip(lax, lbx)]
lcy = [ v[0] - v[1] for v in zip(lay, lby)]
lcz = [ v[0] - v[1] for v in zip(laz, lbz)]

ncx = [ v[0] - v[1] for v in zip(nax, nbx)]
ncy = [ v[0] - v[1] for v in zip(nay, nby)]
ncz = [ v[0] - v[1] for v in zip(naz, nbz)]

# Get size of pairings

cnt_lb = [len(v['unpaired_before']) for k,v in sl.items() if v['type'] == 'learner']
cnt_la = [len(v['unpaired_after']) for k,v in sl.items() if v['type'] == 'learner']
cnt_nb = [len(v['unpaired_before']) for k,v in sl.items() if v['type'] == 'nonlearner']
cnt_na = [len(v['unpaired_after']) for k,v in sl.items() if v['type'] == 'nonlearner']

In [ ]:
#Plot the centroids

learner_bcentroids = go.Scatter3d(
        x=lbx, y=lby, z=lbz,
        mode='markers',
        name='Learner Before centroids',
        text = [ 'Study {0}\nSize: {1}'.format(v[0], v[1]) for v in zip(learners,cnt_lb)],
        marker=dict(
            size=5,
            color='rgb(255,0,0)',
            line=dict(
                color='rgb(255,0,0)',
                width=1
            ),
        opacity=0.5
        )
    )

learner_acentroids = go.Scatter3d(
        x=lax, y=lay, z=laz,
        mode='markers',
        name='Learner After centroids',
        text = [ 'Study {0}\\nSize: {1}'.format(v[0], v[1]) for v in zip(learners,cnt_la)],
        marker=dict(
            size=5,
            color='rgb(0,255,0)',
            line=dict(
                color='rgb(0,255,0)',
                width=1
            ),
        opacity=0.5
        )
    )

nonlearner_bcentroids = go.Scatter3d(
        x=nbx, y=nby, z=nbz,
        mode='markers',
        name='Nonlearner Before centroids',
        text = [ 'Study: {0}\nSize: {1}'.format(v[0], v[1]) for v in zip(nonlearners,cnt_nb)],
        marker=dict(
            size=5,
            color='rgb(0,0,255)',
            line=dict(
                color='rgb(0,0,255)',
                width=.5
            ),
        opacity=0.5
        )
    )

nonlearner_acentroids = go.Scatter3d(
        x=nax, y=nay, z=naz,
        mode='markers',
        name='Nonlearner After centroids',
        text = [ 'Study: {0}\nSize: {1}'.format(v[0], v[1]) for v in zip(nonlearners,cnt_na)],
        marker=dict(
            size=5,
            color='rgb(128,128,128)',
            line=dict(
                color='rgb(128,128,128)',
                width=.5
            ),
        opacity=0.5
        )
    )

data = [learner_bcentroids, learner_acentroids, nonlearner_acentroids, nonlearner_bcentroids]

layout = go.Layout(
            height=1000, width=1000,
            title='Centroid Locations',
        )

fig = dict(data=data, layout=layout)
    
py.offline.iplot(fig, filename='unpaired-centroid')



In [ ]:
#Plot the centroids

learner_vcentroids = go.Scatter3d(
        x=lcx, y=lcy, z=lcz,
        mode='markers',
        name='Learner Centroid Vectors',
        text=learners,
        marker=dict(
            size=5,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.5
        )
    )

nonlearner_vcentroids = go.Scatter3d(
        x=ncx, y=ncy, z=ncz,
        mode='markers',
        name='Nonlearner Centroid Vectors',
        text=nonlearners,
        marker=dict(
            size=5,
            line=dict(
            color='rgb(0,0,255)',
            width=0.5
            ),
        opacity=0.5
        )
    )


data = [learner_vcentroids, nonlearner_vcentroids]
layout = go.Layout(
            height=1000, width=1000,
            title='Centroid Vectors',
        )

fig = dict(data=data, layout=layout)
    
py.offline.iplot(fig, filename='centroid-vectors')




<H1> Paired and Unpaired Synapses

In [ ]:
def plot_pairs(study_list):

    data = []
    steps = []
    
    for s in study_list:
        before = s['paired_before']
        after = s['paired_after']
        up_before = s['unpaired_before']
        up_after = s['unpaired_after']
  
        before_synapses = go.Scatter3d(
            visible = False,
            x=before['x'], y=before['y'], z=before['z'],
            legendgroup='paired',
            name='Before Paired {0} (Rad {1})'.format(s['study'], s['radius']),
            mode='markers',
            marker=dict(
                size=2,
                line=dict(
                color='rgb(0,255,0)',
                width=0.5
            ),
            opacity=0.5
            )
        )

        after_synapses = go.Scatter3d(
            visible = False,
            x=after['x'], y=after['y'], z=after['z'],
            legendgroup='paired',
            name='After Paired {0} (Rad {1})'.format(s['study'], s['radius']),
            mode='markers',
            marker=dict(
                size=2,
                line=dict(
                    color='rgb(255,0,0)',
                    width=.5
                ),
            opacity=0.5
            )
        )
    
        up_before_synapses = go.Scatter3d(
            visible = False,      
            x=up_before['x'], y=up_before['y'], z=up_before['z'],
            legendgroup='unpaired',
            name='Before Unpaired {0} (Rad {1})'.format(s['study'], s['radius']),
            mode='markers',
            marker=dict(
                size=2,
                line=dict(
                color='rgb(0,0,255)',
                width=0.5
                ),
            opacity=0.5
            )
        )

        up_after_synapses = go.Scatter3d(
            visible = False,
            x=up_after['x'], y=up_after['y'], z=up_after['z'],
            legendgroup='unpaired',
            name='After Unpaired {0} (Rad {1})'.format(s['study'], s['radius']),
            mode='markers',
            marker=dict(
                size=2,
                line=dict(
                    color='rgb(128,128,128)',
                    width=.5
                ),
            opacity=0.5
            )
        )
        
        data.extend([ before_synapses, after_synapses, up_before_synapses, up_after_synapses])
        steps.append(dict(label='{0} ({1})'.format(s['study'],s['radius'])))
    
    # Set first study to be visible
    for i in range(4):
        data[i]['visible'] = True
        
    for i,step in enumerate(steps):
        l = ['visible', [False] * len(data)]
        l[1][i*4:i*4+4] = [True] * 4
        step['args'] = l
        step['method'] = 'restyle'
#        step['args'][i*4:i*4 +4] = [True] * 4 # Toggle i'th trace to "visible"
        
    sliders = [dict(
        active = 0,
        currentvalue = {"visible": False},
        pad = {"t": 50},
        steps = steps
    )]

    layout = go.Layout(
            height=1000, width=1000,
            title='Paired and Unpaired Synapapses',
            sliders=sliders
        )

    fig = dict(data=data, layout=layout)
    py.offline.iplot(fig, filename='Synapse Pair Distance')




In [ ]:
# Get all radii for a specific study
s_list = [v[plot_study] for k,v in pairs.items() ]


s_list = [ v for k,v in pairs[2.0].items() if v['type'] == 'learner']

print('Plotting {0} studies'.format(len(s_list)))
plot_pairs(s_list)
    